# Tool 3 - Quality Validator (Hybrid Pattern)

**Status:** ✅ Ready for Databricks | **LLM Cost:** $0 (deterministic) or ~$0.002 (hybrid) | **Performance:** <1s or ~10s

**Pattern:** Deterministic coverage checks + optional LLM enhancement

**Showcase:** Hybrid validation - fast deterministic metrics + smart LLM risk assessment (configurable).

**Key Features:**
- Hybrid async function with configurable LLM fallback (`use_llm_enhancement=True/False`)
- Deterministic coverage metrics: description, owner, source (always runs, <1s)
- Optional LLM enhancement: risk level (LOW/MEDIUM/HIGH/CRITICAL), text quality, P0/P1/P2 recommendations
- Anomaly detection: orphan entities, missing owners, suspicious patterns
- Two modes: `hybrid` (default) or `deterministic_only`

**TODO:**
- [ ] Add lineage coverage metric (upstream/downstream sources)
- [ ] Validate recommendation priority distribution (not all P0)
- [ ] Test with low-quality metadata (0% coverage scenarios)
- [ ] Add trend analysis (quality over time)

**IDEA:**
- Cache LLM enhancement results for unchanged structures (avoid re-analysis)
- Add custom quality rules engine (user-defined thresholds)
- Export recommendations as actionable Jira/GitHub issues

In [ ]:
# Install dependencies
%pip install pydantic-ai>=0.0.49 pydantic>=2.8.0

In [ ]:
# Restart Python kernel to use new packages
dbutils.library.restartPython()  # type: ignore

In [ ]:
import asyncio
import json
import os
from datetime import datetime
from pydantic import BaseModel, Field
from pydantic_ai import Agent  # type: ignore

In [ ]:
# Configure Azure OpenAI from Databricks secrets
AZURE_ENDPOINT = dbutils.secrets.get(scope="mcop", key="azure-openai-endpoint").strip()  # type: ignore
AZURE_API_KEY = dbutils.secrets.get(scope="mcop", key="azure-openai-api-key").strip()  # type: ignore
DEPLOYMENT_NAME = dbutils.secrets.get(scope="mcop", key="azure-openai-deployment-name").strip()  # type: ignore

# Clean endpoint (remove /openai/v1/ if present - Pydantic AI will handle routing)
azure_endpoint_clean = AZURE_ENDPOINT.replace("/openai/v1/", "").replace("/openai/v1", "").rstrip("/")

# Set environment variables for Pydantic AI (Azure OpenAI compatible)
os.environ["OPENAI_BASE_URL"] = f"{azure_endpoint_clean}/openai/deployments/{DEPLOYMENT_NAME}"
os.environ["OPENAI_API_KEY"] = AZURE_API_KEY

MODEL_NAME = f"openai:{DEPLOYMENT_NAME}"
print(f"✅ Configured model: {MODEL_NAME}")
print(f"   Base URL: {os.environ['OPENAI_BASE_URL']}")


In [ ]:
# ==========================================
# PYDANTIC SCHEMAS - HYBRID QUALITY VALIDATION
# ==========================================
# Tool 3 účel: Validovat metadata kvalitu pomocí HYBRID patternu
# - DETERMINISTIC část: Coverage metrics (vždy běží, <1s)
# - LLM ENHANCEMENT část: Sémantická analýza (optional, ~10s)
#
# Proč hybrid:
# - Deterministické metriky = rychlé, spolehlivé, konzistentní
# - LLM enhancement = kontextové, sémantické, human-like reasoning
# - Use case: Produkce = deterministic only, Review = hybrid

# ==========================================
# SCHEMA 1: Recommendation (doporučení pro zlepšení)
# ==========================================
# Co je Recommendation:
# - Actionable improvement suggestion z LLM
# - Prioritizováno (P0/P1/P2) pro task management
# - Kategorizováno (coverage/consistency/naming/security)
#
# Příklady:
# - P0: "Critical: 5 fact tables missing owners" (musí se vyřešit ASAP)
# - P1: "High: 30% tables lack descriptions" (důležité, ale ne blocker)
# - P2: "Medium: Consider standardizing naming convention" (nice-to-have)
class Recommendation(BaseModel):
    """Quality improvement recommendation (z LLM enhancement).

    Pydantic použití:
    - priority: P0 (critical - blocker), P1 (high - important), P2 (medium - nice-to-have)
    - category: Typ problému
      * coverage: Chybějící metadata (description, owner, source)
      * consistency: Nekonzistentní naming, formatting
      * naming: Naming conventions violations
      * security: Security concerns (PII exposure, access control)
    - message: Actionable text (ne jen "bad quality", ale "Add descriptions to 5 fact tables: ...")
    """
    priority: str = Field(description="P0 (critical), P1 (high), P2 (medium)")
    category: str = Field(description="Category: coverage, consistency, naming, security")
    message: str = Field(description="Actionable recommendation text")

# ==========================================
# SCHEMA 2: AnomalyNote (datová anomálie)
# ==========================================
# Co je AnomalyNote:
# - Detekovaná data quality issue (z LLM analýzy)
# - Suspicious patterns (orphan entities, missing owners, weird naming)
# - Severity-based prioritization
#
# Příklady:
# - "Orphan entity: customer_backup table (no FK relationships, no owner)"
# - "Missing owner: 3 dimension tables have NULL owner_id"
# - "Suspicious pattern: tmp_data_2023 (temp table in production?)"
class AnomalyNote(BaseModel):
    """Data quality anomaly (detekovaná z LLM analýzy).

    Pydantic použití:
    - entity: Která entita/tabulka má problém
    - anomaly_type: Typ anomálie
      * orphan_entity: Entity bez vztahů (izolovaná tabulka)
      * missing_owner: Entity bez vlastníka (governance risk)
      * suspicious_pattern: Weird naming/structure (tmp_, backup_, test_)
    - severity: high (must fix), medium (should review), low (nice-to-know)
    - details: Konkrétní popis (ne jen "anomaly found", ale "Table has 0 FK, created 2y ago, no usage")
    """
    entity: str = Field(description="Entity or table name with anomaly")
    anomaly_type: str = Field(description="Type: orphan_entity, missing_owner, suspicious_pattern")
    severity: str = Field(description="Severity: high, medium, low")
    details: str = Field(description="Anomaly details")

# ==========================================
# SCHEMA 3: LLMEnhancement (LLM sémantická analýza)
# ==========================================
# Co je LLMEnhancement:
# - ROOT MODEL pro enhancement_agent (result_type)
# - Sémantická analýza nad deterministickými metrikami
# - Human-like reasoning (risk assessment, text quality, anomalies)
#
# Proč LLM:
# - Risk level vyžaduje kontext (30% missing owners = HIGH or MEDIUM? Depends on entity criticality)
# - Text quality scoring = sémantická analýza (není jen "is description present", ale "is it useful?")
# - Anomaly detection = pattern recognition (LLM umí najít suspicious patterns)
class LLMEnhancement(BaseModel):
    """LLM-enhanced quality assessment (optional, ~10s).

    Pydantic použití:
    - risk_level: Aggregovaný risk assessment
      * CRITICAL: Major governance gaps (>50% missing owners, 0 descriptions)
      * HIGH: Significant issues (20-50% coverage gaps)
      * MEDIUM: Minor issues (5-20% gaps)
      * LOW: Good quality (<5% gaps)
    - text_quality_score: Sémantická kvalita textů (0.0-1.0)
      * 1.0 = všechny descriptions jsou detailed, actionable, up-to-date
      * 0.5 = descriptions jsou generic, outdated, nebo incomplete
      * 0.0 = missing descriptions nebo complete gibberish
    - recommendations: List[Recommendation] prioritizovaných akcí (P0/P1/P2)
    - anomalies: List[AnomalyNote] detekovaných weird patterns
    - summary: Executive summary pro stakeholders (1-2 věty)
    """
    risk_level: str = Field(description="Overall risk: LOW, MEDIUM, HIGH, CRITICAL")
    text_quality_score: float = Field(description="Text quality 0-1")
    recommendations: list[Recommendation] = Field(description="P0/P1/P2 recommendations")
    anomalies: list[AnomalyNote] = Field(default_factory=list, description="Detected anomalies")
    summary: str = Field(description="Executive summary")

# ==========================================
# SCHEMA 4: CoverageMetrics (deterministické metriky)
# ==========================================
# Co je CoverageMetrics:
# - Rychlé (<1s) coverage statistics
# - Fraction-based (0.0-1.0) pro easy comparison
# - VŽDY běží (i když LLM enhancement = False)
#
# Proč deterministické:
# - Consistency: Vždy stejný výsledek pro stejný input
# - Speed: <1s (no LLM call)
# - Auditability: Clear calculation logic
class CoverageMetrics(BaseModel):
    """Deterministic coverage metrics (VŽDY běží, <1s).

    Pydantic použití:
    - description_coverage: Kolik % entit má description
      Calculation: entities_with_description / total_entities
      Example: 8/10 tables have description → 0.8
    - owner_coverage: Kolik % entit má owner_id
      Example: 5/10 tables have owner → 0.5
    - source_coverage: Kolik % entit má source system info
      Example: 10/10 tables have source → 1.0
    - total_entities: Počet evaluovaných entit (for context)
    - timestamp: ISO 8601 timestamp (pro audit trail)
    """
    description_coverage: float = Field(description="Fraction of entities with descriptions")
    owner_coverage: float = Field(description="Fraction of entities with owners")
    source_coverage: float = Field(description="Fraction of entities with sources")
    total_entities: int = Field(description="Total entities evaluated")
    timestamp: str = Field(description="ISO 8601 timestamp")

# ==========================================
# SCHEMA 5: QualityReport (ROOT MODEL pro validate_quality function)
# ==========================================
# Co je QualityReport:
# - Kompletní quality validation výsledek
# - HYBRID: coverage (always) + llm_enhancement (optional)
# - Execution time tracking (performance monitoring)
#
# Design pattern:
# - coverage: CoverageMetrics (required, always present)
# - llm_enhancement: LLMEnhancement | None (optional, depends on use_llm_enhancement flag)
# - mode: "hybrid" or "deterministic_only" (pro audit trail)
class QualityReport(BaseModel):
    """Complete quality validation report (hybrid pattern).

    Pydantic použití:
    - coverage: CoverageMetrics (VŽDY přítomno, rychlé metriky)
    - llm_enhancement: LLMEnhancement | None (optional)
      * None = use_llm_enhancement=False (produkce, rychlý check)
      * LLMEnhancement object = use_llm_enhancement=True (review, detailní analýza)
    - execution_time_seconds: Performance tracking
      * deterministic_only: <1s
      * hybrid: ~10s (depends on LLM latency)
    - mode: Which mode was used (pro audit trail)

    Use cases:
    - Produkce CI/CD: QualityReport(coverage=..., llm_enhancement=None, mode="deterministic_only")
      → Rychlý check, <1s, konzistentní
    - Manual review: QualityReport(coverage=..., llm_enhancement=..., mode="hybrid")
      → Detailní analýza, ~10s, sémantické reasoning
    """
    coverage: CoverageMetrics
    llm_enhancement: LLMEnhancement | None = None
    execution_time_seconds: float
    mode: str = Field(description="hybrid or deterministic_only")

In [ ]:
# ==========================================
# PYDANTIC AI AGENT - enhancement_agent
# ==========================================
# Účel: Optional LLM agent pro sémantickou kvalitu analýzu
# Input: coverage metrics + structure JSON
# Output: LLMEnhancement (risk_level, text_quality_score, recommendations, anomalies, summary)
#
# Proč OPTIONAL (ne always-on):
# - Latency: ~10s per run (LLM call overhead)
# - Cost: API charges per request
# - Deterministic metrics často stačí (CI/CD pipelines)
# - LLM enhancement = for deep reviews, manual audits
#
# Kdy použít LLM enhancement:
# - ✅ Manual metadata review sessions
# - ✅ Quarterly governance audits
# - ✅ Pre-production validation (before release)
# - ❌ CI/CD automated checks (use deterministic only)
# - ❌ Real-time dashboards (too slow)

enhancement_agent = Agent(
    MODEL_NAME,                          # "openai:test-gpt-5-mini"
    result_type=LLMEnhancement,          # LLM MUSÍ vrátit LLMEnhancement object
    system_prompt="""You are a metadata quality expert.

Analyze the provided structure and assess:
1. Risk level (LOW/MEDIUM/HIGH/CRITICAL) based on coverage gaps
2. Text quality score (0-1) for descriptions and documentation
3. Prioritized recommendations (P0=critical, P1=high, P2=medium)
4. Data quality anomalies (orphan entities, missing owners, suspicious patterns)
5. Executive summary for stakeholders

Be specific and actionable."""
)
# System prompt strategie:
# 1. Risk level: Kontextová analýza (30% missing owners = HIGH if critical entities)
# 2. Text quality: Sémantické scoring (ne jen "present/absent", ale "useful/useless")
# 3. Recommendations: Prioritizace (P0 = must fix now, P1 = important, P2 = nice-to-have)
# 4. Anomalies: Pattern recognition (orphan tables, suspicious naming, missing governance)
# 5. Summary: Executive-friendly (non-technical language)
#
# LLM dostane:
# - System prompt (výše)
# - User prompt (coverage metrics + structure JSON)
# - JSON Schema pro LLMEnhancement (automaticky přidáno Pydantic AI)
#
# LLM vrátí:
# {
#   "risk_level": "HIGH",
#   "text_quality_score": 0.6,
#   "recommendations": [
#     {"priority": "P0", "category": "coverage", "message": "Add owners to 5 fact tables"},
#     {"priority": "P1", "category": "consistency", "message": "Standardize naming: use snake_case"}
#   ],
#   "anomalies": [
#     {"entity": "tmp_data_2023", "anomaly_type": "suspicious_pattern", "severity": "medium", "details": "Temp table in prod?"}
#   ],
#   "summary": "Metadata quality has significant gaps (HIGH risk). Focus on owner coverage (50% missing)."
# }

print("✅ Enhancement agent created")

In [ ]:
# ==========================================
# HYBRID VALIDATION FUNCTION - validate_quality()
# ==========================================
# Tool 3 HYBRID PATTERN:
# - Step 1: DETERMINISTIC coverage metrics (VŽDY běží, <1s)
# - Step 2: OPTIONAL LLM enhancement (pokud use_llm_enhancement=True, ~10s)
#
# Proč hybrid design:
# - Flexibility: Můžeš zvolit speed vs depth
# - Cost optimization: LLM enhancement jen když potřebuješ
# - Consistency: Deterministické metriky vždy stejné
# - Human-like reasoning: LLM enhancement pro context-aware analysis
#
# Use case matrix:
# | Scenario                  | use_llm_enhancement | Execution time | Output                    |
# |---------------------------|---------------------|----------------|---------------------------|
# | CI/CD automated check     | False               | <1s            | Coverage metrics only     |
# | Manual review session     | True                | ~10s           | Coverage + LLM insights   |
# | Real-time dashboard       | False               | <1s            | Fast metrics              |
# | Quarterly governance audit| True                | ~10s           | Deep analysis + actions   |

async def validate_quality(structure: dict, use_llm_enhancement: bool = True) -> QualityReport:
    """Hybrid quality validation: deterministic coverage + optional LLM enhancement.

    Args:
        structure: Tool 2 structural classification output
            Obsahuje: facts[], dimensions[], relationships[], metrics

        use_llm_enhancement: If True, runs LLM analysis (~10s). If False, only coverage (<1s).
            Default: True (comprehensive analysis)
            Set False: For CI/CD, real-time checks, cost optimization

    Returns:
        QualityReport:
        - coverage: CoverageMetrics (VŽDY přítomno)
        - llm_enhancement: LLMEnhancement | None (depends on use_llm_enhancement)
        - execution_time_seconds: Performance tracking
        - mode: "hybrid" or "deterministic_only"

    Workflow:
        Step 1: Calculate deterministic coverage metrics (<1s)
            - Count entities with description/owner/source
            - Calculate fractions (0.0-1.0)
            - Create CoverageMetrics object

        Step 2: Optionally run LLM enhancement (~10s)
            - IF use_llm_enhancement=True:
                * Prepare prompt (coverage + structure)
                * Call enhancement_agent
                * Get LLMEnhancement object (risk, quality score, recommendations, anomalies)
            - ELSE:
                * Skip LLM call
                * Set llm_enhancement=None

        Step 3: Return QualityReport (combine deterministic + optional LLM)
    """

    start_time = datetime.now()

    # ==========================================
    # STEP 1: Deterministic coverage (VŽDY běží)
    # ==========================================
    # Rychlé metriky (<1s):
    # - description_coverage = kolik % entit má description field
    # - owner_coverage = kolik % entit má owner field
    # - source_coverage = kolik % entit má source field
    #
    # Algorithm:
    # 1. Merge facts + dimensions do all_entities list
    # 2. Count entities where field is present (e.get("description") returns truthy value)
    # 3. Calculate fraction = count / total (0.0 if total=0 to avoid division by zero)

    all_entities = structure.get("facts", []) + structure.get("dimensions", [])
    total = len(all_entities)

    description_count = sum(1 for e in all_entities if e.get("description"))
    owner_count = sum(1 for e in all_entities if e.get("owner"))
    source_count = sum(1 for e in all_entities if e.get("source"))

    coverage = CoverageMetrics(
        description_coverage=description_count / total if total > 0 else 0.0,
        owner_coverage=owner_count / total if total > 0 else 0.0,
        source_coverage=source_count / total if total > 0 else 0.0,
        total_entities=total,
        timestamp=datetime.now().isoformat()
    )
    # Deterministický output:
    # CoverageMetrics(
    #   description_coverage=0.8,  # 8/10 entities have description
    #   owner_coverage=0.5,        # 5/10 entities have owner
    #   source_coverage=1.0,       # 10/10 entities have source
    #   total_entities=10,
    #   timestamp="2025-11-10T14:30:00Z"
    # )

    # ==========================================
    # STEP 2: Optional LLM enhancement
    # ==========================================
    # IF use_llm_enhancement=True:
    #   - Připrav prompt s coverage metrics + structure JSON
    #   - Zavolej enhancement_agent (LLM call, ~10s)
    #   - Get LLMEnhancement object (risk_level, text_quality_score, recommendations, anomalies, summary)
    # ELSE:
    #   - Skip LLM call (save time & cost)
    #   - Set llm_result=None

    llm_result = None
    if use_llm_enhancement:
        prompt = f"""Analyze this metadata structure:

Coverage Metrics:
- Description coverage: {coverage.description_coverage:.1%}
- Owner coverage: {coverage.owner_coverage:.1%}
- Source coverage: {coverage.source_coverage:.1%}

Structure:
{json.dumps(structure, indent=2)}

Provide risk assessment, quality score, recommendations (P0/P1/P2), anomalies, and summary."""
        # Prompt strategie:
        # - Coverage metrics first (quick overview pro LLM)
        # - Structure JSON second (full context)
        # - Explicit ask for output fields (risk, quality, recommendations, anomalies, summary)

        result = await enhancement_agent.run(prompt)
        llm_result = result.data  # Type: LLMEnhancement
        # LLM output example:
        # LLMEnhancement(
        #   risk_level="HIGH",
        #   text_quality_score=0.6,
        #   recommendations=[
        #     Recommendation(priority="P0", category="coverage", message="Add owners to 5 fact tables"),
        #     Recommendation(priority="P1", category="consistency", message="Standardize naming")
        #   ],
        #   anomalies=[
        #     AnomalyNote(entity="tmp_data", anomaly_type="suspicious_pattern", severity="medium", details="Temp table in prod")
        #   ],
        #   summary="HIGH risk due to 50% missing owners. Focus on governance gaps."
        # )

    # ==========================================
    # STEP 3: Calculate execution time & mode
    # ==========================================
    execution_time = (datetime.now() - start_time).total_seconds()
    mode = "hybrid" if use_llm_enhancement else "deterministic_only"
    # Execution time examples:
    # - deterministic_only: ~0.5s (just coverage calculation)
    # - hybrid: ~10s (coverage + LLM call + validation)

    # ==========================================
    # STEP 4: Return QualityReport
    # ==========================================
    return QualityReport(
        coverage=coverage,                      # CoverageMetrics (VŽDY přítomno)
        llm_enhancement=llm_result,             # LLMEnhancement | None (depends on flag)
        execution_time_seconds=execution_time,  # Performance tracking
        mode=mode                               # "hybrid" or "deterministic_only"
    )
    # QualityReport usage:
    # - report.coverage.description_coverage → deterministická metrika
    # - report.llm_enhancement.risk_level → LLM assessment (if present)
    # - report.llm_enhancement.recommendations → actionable P0/P1/P2 items (if present)

print("✅ Hybrid validation function defined")

In [ ]:
# Load Tool 2 structure from DBFS
structure_path = "/dbfs/FileStore/mcop/tool2/structure.json"

with open(structure_path, "r") as f:
    structure = json.load(f)

print(f"✅ Loaded structure from: {structure_path}")
print(f"   Facts: {len(structure.get('facts', []))}")
print(f"   Dimensions: {len(structure.get('dimensions', []))}")

In [ ]:
# Run hybrid validation (deterministic + LLM)
report = await validate_quality(structure, use_llm_enhancement=True)

print(f"\n✅ Validation complete ({report.mode} mode)")
print(f"   Execution time: {report.execution_time_seconds:.2f}s")
print(f"\n📊 Coverage Metrics:")
print(f"   Description: {report.coverage.description_coverage:.1%}")
print(f"   Owner: {report.coverage.owner_coverage:.1%}")
print(f"   Source: {report.coverage.source_coverage:.1%}")

if report.llm_enhancement:
    print(f"\n🤖 LLM Enhancement:")
    print(f"   Risk Level: {report.llm_enhancement.risk_level}")
    print(f"   Text Quality: {report.llm_enhancement.text_quality_score:.2f}")
    print(f"   Recommendations: {len(report.llm_enhancement.recommendations)}")
    print(f"   Anomalies: {len(report.llm_enhancement.anomalies)}")

In [ ]:
# Save quality report to DBFS
output_path = "/dbfs/FileStore/mcop/tool3/quality_report.json"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, "w") as f:
    json.dump(report.model_dump(), f, indent=2)

print(f"✅ Quality report saved: {output_path}")

In [ ]:
# Display detailed report
print("\n" + "="*80)
print("QUALITY VALIDATION REPORT")
print("="*80)

print(f"\n📈 Coverage Summary:")
print(f"   Total entities: {report.coverage.total_entities}")
print(f"   Description coverage: {report.coverage.description_coverage:.1%}")
print(f"   Owner coverage: {report.coverage.owner_coverage:.1%}")
print(f"   Source coverage: {report.coverage.source_coverage:.1%}")

if report.llm_enhancement:
    print(f"\n🔍 Risk Assessment:")
    print(f"   Risk Level: {report.llm_enhancement.risk_level}")
    print(f"   Text Quality: {report.llm_enhancement.text_quality_score:.2f}")

    print(f"\n💡 Recommendations ({len(report.llm_enhancement.recommendations)}):")
    for rec in report.llm_enhancement.recommendations[:5]:  # Top 5
        print(f"   [{rec.priority}] {rec.category}: {rec.message}")

    print(f"\n⚠️  Anomalies ({len(report.llm_enhancement.anomalies)}):")
    for anomaly in report.llm_enhancement.anomalies[:3]:  # Top 3
        print(f"   [{anomaly.severity.upper()}] {anomaly.anomaly_type}: {anomaly.details}")

    print(f"\n📝 Executive Summary:")
    print(f"   {report.llm_enhancement.summary}")

print(f"\n⏱️  Execution: {report.execution_time_seconds:.2f}s ({report.mode} mode)")
print("="*80)